# DATA COLLECTION

In [1]:
# Importing the necessary packages
import sklearn
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [2]:
def data_gather(url1):
    homepage_source=urlopen(url1).read()
    homepage_soup=BeautifulSoup(homepage_source,'lxml')
    #rating=[]
    text=[]
    review_class=[]
    #count=0
    for url in homepage_soup.find_all('a'):
        #print(url.get('href'))
        #count=count+1
        #print(count)
        if url.get('href')=="index.html":
            continue
        source=urlopen('http://mlg.ucd.ie/modules/yalp/'+url.get('href')).read()
        soup=BeautifulSoup(source,'lxml')
        for reviews in soup.find_all("div",{"class":"review"}):
            a=reviews.find_all("p",{"class":"rating"})[0].find_all('img')[0]["alt"]
            #rating.append(a)
            if (a=='1-star'or a=='2-star'or a=='3-star'):
                review_class.append('Negative')
            else:
                review_class.append('Postive')
    
            text.append(reviews.find_all("p",{"class":"review-text"})[0].text)
        
        
    df = pd.DataFrame(list(zip(text,review_class)), 
               columns =['Review','Class']) 

    return df

In [54]:
df_cafe=data_gather('http://mlg.ucd.ie/modules/yalp/cafes_list.html')
df_cafe.to_pickle('cafe_data.pkl')
df_cafe.tail(10)

,Review,Class
1990,"Went there for lunch and had the a few apps, t...",Negative
1991,I've only been here once but the sandwiches ar...,Postive
1992,Came here for a birthday lunch with a friend. ...,Postive
1993,Conveniently located and with such good lookin...,Negative
1994,"Their frozen yogurt is to die for. Seriously, ...",Postive
1995,I hate to be one of those obnoxious people who...,Negative
1996,It's always a stop here for me either for a qu...,Negative
1997,it is nice to go there if you'd like to go sho...,Negative
1998,My girlfriend and I had lunch there a few days...,Negative
1999,"If you are ever craving a pick and choose, caf...",Postive


In [41]:
df_hotel=data_gather('http://mlg.ucd.ie/modules/yalp/hotels_list.html')
df_hotel.to_pickle('hotel_data.pkl')
df_hotel.head()

,Review,Class
0,Melissa took us on a tour of Asia in the space...,Postive
1,With a group of seven of us visiting Montreal ...,Postive
2,Melissa is a gem! My fiancé found her tour on ...,Postive
3,A perfect day in Montreal! Melissa outfitted u...,Postive
4,I had a really great food truck tour with Meli...,Postive


In [42]:
df_restaurant=data_gather('http://mlg.ucd.ie/modules/yalp/restaurants_list.html')
df_restaurant.to_pickle('restaurant_data.pkl')
df_restaurant.head()

,Review,Class
0,My husband and I had a rare afternoon off so w...,Negative
1,For years I thought this was only a wine store...,Postive
2,This place is so charming! I went with my husb...,Postive
3,We have been wanting to try this place for a c...,Postive
4,Decor looks ok but layout is too busy. Difficu...,Negative


# DATA PREPROCESSING

In [6]:
import nltk
import re
import string
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
unclean_cafe= pd.read_pickle('cafe_data.pkl')
unclean_hotel= pd.read_pickle('hotel_data.pkl')
unclean_restaurant= pd.read_pickle('restaurant_data.pkl')

In [45]:
unclean_cafe

,Review,Class
0,Pros: Lots of items you would not expect from ...,Postive
1,Best egg-tarts in town! There's really not muc...,Postive
2,I've been to ABC Bakery a few times since I re...,Negative
3,"FYI, Closed Monday's New ownership for about 1...",Negative
4,The inside may not look like much but they mak...,Postive
5,I usually come for the egg tart and it is one ...,Postive
6,"Everything, I meant everything were good. Were...",Postive
7,"It's more like 3 and a 1/2 stars, but if value...",Negative
8,This place got to have the best Hong Kong styl...,Postive
9,Make a quick stop here while waiting for dim s...,Postive


In [8]:
def cleaning(review):
    #lowercase
    review = review.lower()
    #punctuations
    #review = re.sub('[%s]' % re.escape(string.punctuation), '', review)
    #numbers
    #review = re.sub('\w*\d\w*', '', review)
    
    #line space
    review = re.sub('\n', '', review)
    #only alpahbets are retained
    review = re.sub('[^a-z\s]+',' ',review)
    #multiple spaces by single space
    review = re.sub('(\s+)',' ',review)
    
    return review



In [9]:
#Tokenization
def tokenize(df_clean):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    df_clean['Review']=df_clean.Review.apply(lambda x: w_tokenizer.tokenize(x))

#Stop Words    
def clean_stopwords(review,stop_words1):
    words=[w for w in review if w not in stop_words1]
    return words

#Lemmatization
def clean_lemmatize(review,lemmatizer):
    lem_review=" ".join([lemmatizer.lemmatize(i) for i in review])
    return lem_review

In [10]:
def unclean_to_clean(df):
    df["Clean"]= 'a'
    
    
    for i in range(0,len(df)):
        df["Clean"][i]=cleaning(df["Review"][i])
    
    df_clean=df.filter(['Clean','Class'],axis=1)
    df_clean=df_clean.rename(columns={'Clean':'Review'})
    
    tokenize(df_clean)
    
    from nltk.corpus import stopwords
    stop_words1=stopwords.words('english')
    stop_words1.remove("not")
    df_clean['Review']=df_clean.Review.apply(lambda x: clean_stopwords(x,stop_words1))
    
    lemmatizer = nltk.stem.WordNetLemmatizer()
    df_clean['Review']=df_clean.Review.apply(lambda x: clean_lemmatize(x,lemmatizer))
    
    return df_clean

In [46]:
import nltk
nltk.download('wordnet')

clean_cafe=unclean_to_clean(unclean_cafe)
#clean_cafe

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vinee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
clean_hotel=unclean_to_clean(unclean_hotel)
#clean_hotel

In [48]:
clean_restaurant=unclean_to_clean(unclean_restaurant)
#clean_restaurant

## CATEGORY: CAFE & HOTEL

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [30]:
cv = TfidfVectorizer()
cafe_cv = cv.fit(clean_cafe['Review'])
trans_a = cafe_cv.transform(clean_cafe['Review'])
trans_b = cafe_cv.transform(clean_hotel['Review'])


cafe_matrix = pd.DataFrame(trans_a.toarray(), columns=cv.get_feature_names())
#cafe_matrix.index = clean_cafe.index

hotel_matrix = pd.DataFrame(trans_b.toarray(), columns=cv.get_feature_names())
#hotel_matrix.index = clean_hotel.index


In [52]:
clean_cafe['Review'][1]

'best egg tart town really not much say hong kong style fastfood restaurant come customer service come luxurious architecture come first date come cheap authentic hong kong dish bakery fresh egg tart'

In [32]:
hotel_matrix.head()

,aaa,aaand,aau,ab,abbot,abc,aber,aberration,abhorrent,ability,...,zest,zine,zines,zing,zingy,zomg,zone,zucchini,zuchinni,zupas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
X_train=cafe_matrix
y_train=clean_cafe['Class']
X_test=hotel_matrix
y_test=clean_hotel['Class']

In [36]:
LR= LogisticRegression()
LR.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [37]:
y_predict=LR.predict(X_test)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
al=accuracy_score(y_test,y_predict)
al

0.8135

## CATEGORY: CAFE & RESTAURANT

In [29]:
cv = CountVectorizer()
cafe_cv = cv.fit(clean_cafe['Review'])
trans_a = cafe_cv.transform(clean_cafe['Review'])
trans_c = cafe_cv.transform(clean_restaurant['Review'])


cafe_matrix = pd.DataFrame(trans_a.toarray(), columns=cv.get_feature_names())
cafe_matrix.index = clean_cafe.index

restaurant_matrix = pd.DataFrame(trans_c.toarray(), columns=cv.get_feature_names())
restaurant_matrix.index = clean_restaurant.index

In [30]:
X_train=cafe_matrix
y_train=clean_cafe['Class']
X_test=restaurant_matrix
y_test=clean_restaurant['Class']

In [31]:
LR= LogisticRegression()
LR.fit(X_train,y_train)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
y_predict=LR.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
al=accuracy_score(y_test,y_predict)
al

0.857